In [5]:
import numpy as np
import pandas as pd
import polars as pl
import pytomlpp as toml
from data_preprocessing import getXY, data_normalizeation
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from tqdm import tqdm

In [6]:
cfg = toml.load('/home/gyli/GP_HP/cfg/feature_selection.toml')
data_path = cfg['data_path']
variance_filter_thres = cfg['variance_filter_thres']
raw_x, raw_y = getXY(data_path)
# raw_x = data_normalizeation(raw_x)
future_names = raw_x.columns
pd_x, pd_y = raw_x.to_pandas(), raw_y.to_pandas()
svm_clf = SVC()
X, Y = raw_x.to_pandas(), np.ravel(raw_y.to_pandas())

In [7]:
mb = ['白蛋白', '活化部分凝血活酶时间', '低密度脂蛋白', '钠', '中性粒细胞数', '甘油三酯', '凝血酶原时间']
t_x = X[mb]

In [11]:
gamma = cfg['gamma']
C = cfg['C']
best_score = 0.0
best_params = {'C' : -1.0, 'gamma' : -1.0}
for g in tqdm(gamma, desc='Outer Loop'):
    for c in tqdm(C, desc='Inner Loop'):
        svm = SVC(kernel='linear',gamma=g, C=c)
        # svm.fit(X_SFS1, Y)
        scores = cross_val_score(svm, t_x, Y, cv=5, scoring='accuracy')
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_params['C'] = c
            best_params['gamma'] = g
        tqdm.write(f'finish one param part, c = {c}, gamma = {g}, score = {score}')
tqdm.write(f'best param: C = {best_params["C"]}, gamma = {best_params["gamma"]}, score = {best_score}')

                                                  
Outer Loop:   0%|          | 0/10 [01:09<?, ?it/s]

finish one param part, c = 1, gamma = 1.0, score = 0.7258172408060409


                                                  
Outer Loop:   0%|          | 0/10 [02:49<?, ?it/s]        

finish one param part, c = 2, gamma = 1.0, score = 0.726099044348714
